In [4]:
import copy
import random
import json
import os
import openai

class Debt:
    def __init__(self, name, balance, interest_rate, minimum_payment):
        self.name = name
        self.balance = balance
        self.interest_rate = interest_rate  # Annual rate, e.g., 0.18 for 18%
        self.minimum_payment = minimum_payment

def calculate_consolidated_loan_payment(total_balance, new_interest_rate, term_months):
    """Calculate monthly payment for a consolidated loan using the loan amortization formula."""
    monthly_rate = new_interest_rate / 12
    payment = total_balance * (monthly_rate * (1 + monthly_rate) ** term_months) / ((1 + monthly_rate) ** term_months - 1)
    return payment

def simulate_savings_growth(savings, cash_flow, interest_rate, months, risk_factor='low'):
    """Simulate savings growth with compound interest, adjusted for risk volatility."""
    monthly_rate = interest_rate / 12
    for _ in range(months):
        if risk_factor == 'high':
            adjusted_rate = monthly_rate * (1 + random.uniform(-0.2, 0.2))
        else:
            adjusted_rate = monthly_rate
        savings += savings * adjusted_rate + cash_flow
    return savings

def simulate_financial_plan(debts, income, non_debt_expenses, initial_savings_balance, 
                            risk_tolerance='medium', savings_percentage=0.2, 
                            savings_options={'savings_account': 0.03, 'stock_market': 0.07, 'aggressive_stocks': 0.10}, 
                            strategy='avalanche', consolidation_rate=0.08, consolidation_term=60):
    """Simulate debt repayment and savings maximization with risk-based investment options."""
    debts = [copy.copy(d) for d in debts]
    months = 0
    savings_balance = initial_savings_balance
    total_interest_paid = 0

    risk_map = {
        'low': ('savings_account', 'low'),
        'medium': ('stock_market', 'medium'),
        'high': ('aggressive_stocks', 'high')
    }
    investment_type, _ = risk_map[risk_tolerance]
    savings_interest_rate = savings_options[investment_type]
    monthly_savings_interest = savings_interest_rate / 12

    monthly_savings_amount = income * savings_percentage

    if not debts or strategy == 'savings_max':
        cash_flow = income - non_debt_expenses
        if cash_flow < monthly_savings_amount:
            monthly_savings_amount = cash_flow
        post_debt_savings = {}
        for option, rate in savings_options.items():
            risk = risk_map['low'][1] if option == 'savings_account' else (
                risk_map['medium'][1] if option == 'stock_market' else risk_map['high'][1])
            post_debt_savings[option.replace('_', ' ').title()] = f"${simulate_savings_growth(initial_savings_balance, cash_flow, rate, 60, risk):.2f}"
        return {
            "Financial Plan": {
                "Strategy Used": strategy.capitalize(),
                "Risk Tolerance": risk_tolerance.capitalize(),
                "Debt Repayment Details": {
                    "Months to Debt-Free": "0 months",
                    "Total Interest Paid": "$0.00"
                },
                "Savings Details": {
                    "Initial Savings Balance": f"${initial_savings_balance:.2f}",
                    "Monthly Savings Allocation": f"${monthly_savings_amount:.2f}",
                    "Savings at Debt-Free": f"${initial_savings_balance:.2f}"
                },
                "Savings Projections": {
                    "5-Year Post-Debt Savings": post_debt_savings
                }
            }
        }

    if strategy == 'consolidation':
        total_balance = sum(d.balance for d in debts)
        monthly_payment = calculate_consolidated_loan_payment(total_balance, consolidation_rate, consolidation_term)
        remaining_balance = total_balance
        monthly_rate = consolidation_rate / 12

        while remaining_balance > 0 and months < consolidation_term:
            interest = remaining_balance * monthly_rate
            total_interest_paid += interest
            remaining_balance += interest - monthly_payment
            if remaining_balance < 0:
                remaining_balance = 0
            cash_flow = income - non_debt_expenses - monthly_payment
            monthly_savings_amount = min(cash_flow * savings_percentage, cash_flow)
            savings_balance += savings_balance * monthly_savings_interest + monthly_savings_amount
            months += 1
        if remaining_balance > 0:
            raise ValueError("Consolidation term too short to pay off consolidated debt.")

    elif strategy == 'high_minimum':
        while debts:
            total_minimum_payments = sum(max(d.minimum_payment, d.balance * 0.05) for d in debts)
            cash_flow = income - non_debt_expenses - total_minimum_payments
            monthly_savings_amount = min(cash_flow * savings_percentage, cash_flow)
            extra_debt_payment = cash_flow - monthly_savings_amount
            if extra_debt_payment < 0:
                raise ValueError("Insufficient cash flow for high minimum payments and savings.")
            for d in debts:
                interest = d.balance * (d.interest_rate / 12)
                d.balance += interest
                total_interest_paid += interest
                d.balance -= max(d.minimum_payment, d.balance * 0.05)
            if debts and extra_debt_payment > 0:
                target_debt = min(debts, key=lambda d: d.balance)
                target_debt.balance -= extra_debt_payment
            savings_balance += savings_balance * monthly_savings_interest + monthly_savings_amount
            debts = [d for d in debts if d.balance > 0]
            months += 1

    else:  # 'avalanche' or 'snowball'
        while debts:
            total_minimum_payments = sum(d.minimum_payment for d in debts)
            cash_flow = income - non_debt_expenses - total_minimum_payments
            monthly_savings_amount = min(cash_flow * savings_percentage, cash_flow)
            extra_debt_payment = cash_flow - monthly_savings_amount
            if extra_debt_payment < 0:
                raise ValueError("Insufficient cash flow to cover savings and debt payments.")
            for d in debts:
                interest = d.balance * (d.interest_rate / 12)
                d.balance += interest
                total_interest_paid += interest
                d.balance -= d.minimum_payment
            target_debt = (max(debts, key=lambda d: d.interest_rate) if strategy == 'avalanche' 
                           else min(debts, key=lambda d: d.balance)) if debts else None
            if target_debt and extra_debt_payment > 0:
                target_debt.balance -= extra_debt_payment
            savings_balance += savings_balance * monthly_savings_interest + monthly_savings_amount
            debts = [d for d in debts if d.balance > 0]
            months += 1

    post_debt_cash_flow = income - non_debt_expenses
    post_debt_savings = {}
    for option, rate in savings_options.items():
        risk = risk_map['low'][1] if option == 'savings_account' else (
            risk_map['medium'][1] if option == 'stock_market' else risk_map['high'][1])
        post_debt_savings[option.replace('_', ' ').title()] = f"${simulate_savings_growth(savings_balance, post_debt_cash_flow, rate, 60, risk):.2f}"

    return {
        "Financial Plan": {
            "Strategy Used": strategy.capitalize(),
            "Risk Tolerance": risk_tolerance.capitalize(),
            "Debt Repayment Details": {
                "Months to Debt-Free": f"{months} months",
                "Total Interest Paid": f"${total_interest_paid:.2f}"
            },
            "Savings Details": {
                "Initial Savings Balance": f"${initial_savings_balance:.2f}",
                "Monthly Savings Allocation": f"${monthly_savings_amount:.2f}",
                "Savings at Debt-Free": f"${savings_balance:.2f}"
            },
            "Savings Projections": {
                "5-Year Post-Debt Savings": post_debt_savings
            }
        }
    }



In [5]:
def generate_financial_report_openai(financial_data):
    """
    Generate a detailed financial report using the OpenAI API.
    This function uses the ChatCompletion endpoint with GPT-3.5-turbo.
    """
    
    prompt = f"""
You are a seasoned financial advisor with expertise in personal finance and wealth management. Your task is to generate a detailed, data-driven financial report using the provided financial data. Use the exact numbers from the data and from common financial formulas to make precise recommendations. The report must include specific computed values (e.g., extra funds allocated to each debt, the estimated number of months/years to become debt-free, and the total interest savings) based on the user's financial situation.

Please structure the report as follows:

1. **Introduction**:
   - Greet the user warmly.
   - Summarize the user's overall financial situation, mentioning total income, total debts, expenses, current savings, and investments.
   - Clearly state the chosen strategy (e.g., "avalanche" for debt repayment) and risk tolerance.
   - Outline what the report will cover.

2. **Debt Repayment Plan**:
   - List each type of debt (e.g., Credit Card, Car Loan) along with its balance, interest rate, and minimum payment.
   - For each debt, compute and display:
     - The extra monthly payment allocated (beyond the minimum) to high-interest debts.
     - An estimated timeline (in months/years) to become debt-free if the plan is followed.
     - The estimated total interest that will be saved (for example, "By allocating extra funds to high-interest debts, you can become debt-free in X years and save approximately $Y in interest").
   - Use exact values computed from the provided numbers (for example, if the user's calculations indicate that extra $150 per month should be allocated to the credit card, display that value).

3. **Current Savings Status**:
   - Detail the current savings balance and the recommended monthly savings allocation.
   - Explain how these figures were derived.
   - Show the exact amount that should be saved monthly relative to income and expenses.

4. **Future Savings Projections and Investment Recommendations**:
   - Provide precise 5-year projections for savings growth based on current savings, monthly contributions, and expected investment returns.
   - Break down the recommended splits among a savings account, stock market investments, and more aggressive options.
   - Include specific numbers (e.g., "Investing 30% of your monthly surplus in stocks could grow your savings to approximately $Z in 5 years").
   - Offer clear, actionable steps on how to build and adjust an investment portfolio.

5. **Conclusion**:
   - Summarize the key recommendations with exact figures.
   - Reiterate the specific steps the user should take immediately (e.g., "Allocate an extra $150 per month to your credit card debt and reduce your minimum payment burden, aiming to be debt-free in 36 months").
   - Encourage the user to review and adjust the plan periodically.
   - End with an encouraging note that highlights financial empowerment.

Ensure that every recommendation is supported by specific numbers computed from the input data, not just general statements. The final report should be comprehensive (around 500 words) and include clear, actionable steps with exact values that the user can directly apply.
"""


    
    messages = [
        {"role": "system", "content": "You are a helpful and knowledgeable financial advisor with extreme financial expertise."},
        {"role": "user", "content": prompt}
    ]
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=1000,
            top_p=0.9,
            frequency_penalty=0,
            presence_penalty=0
        )
        report = response["choices"][0]["message"]["content"].strip()
        return report
    except Exception as e:
        raise Exception(f"Error generating report via OpenAI: {e}")

# Example usage:
if __name__ == "__main__":
    debts = [
        Debt("Credit Card", 5000, 0.18, 100),
        Debt("Car Loan", 10000, 0.05, 200)
    ]
    income = 5000
    non_debt_expenses = 3000
    initial_savings_balance = 2000
    strategy = 'avalanche'
    risk_tolerance = 'medium'
    
    financial_data = simulate_financial_plan(
        debts, income, non_debt_expenses, initial_savings_balance, 
        risk_tolerance=risk_tolerance, strategy=strategy
    )
    
    try:
        report = generate_financial_report_openai(financial_data)
        print("\nGenerated Financial Report:")
        print(report)
    except Exception as e:
        print(f"Error generating report: {e}")


Generated Financial Report:
### Financial Report for [User's Name]

#### 1. Introduction:
Dear [User's Name],

It's my pleasure to provide you with a detailed financial report tailored to your unique situation. Let's start by summarizing your current financial standing:

- Total Income: $X per month
- Total Debts:
  - Credit Card Debt: $Y at an interest rate of Z%
  - Car Loan: $A at an interest rate of B%
- Monthly Expenses: $C
- Current Savings: $D
- Investments: $E
- Chosen Strategy: Avalanche Debt Repayment Method
- Risk Tolerance: Moderate

This report will cover a debt repayment plan focusing on the avalanche method, current savings status, future savings projections, and investment recommendations.

#### 2. Debt Repayment Plan:
- **Credit Card Debt**:
  - Balance: $Y, Interest Rate: Z%, Minimum Payment: $M
  - Extra Monthly Payment: $P
  - Estimated Time to Debt-Free: X years
  - Total Interest Saved: $Q

- **Car Loan**:
  - Balance: $A, Interest Rate: B%, Minimum Payment: $N
 

## Module

In [7]:
import copy
import math
import random

#####################################
# Data Structures and Input Parsing #
#####################################

# Expected input format (example):
# {
#   "income": [{"title": "Job", "amount": 5000}, {"title": "Side Hustle", "amount": 1000}],
#   "debt": [
#       {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
#       {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
#   ],
#   "expenses": {
#       "needs": [{"title": "Rent", "amount": 1500}, {"title": "Utilities", "amount": 300}],
#       "wants": [{"title": "Dining Out", "amount": 200}, {"title": "Entertainment", "amount": 150}]
#   },
#   "savings": 2000
# }

###############################
# Basic Financial Calculations #
###############################

def compute_total_income(income_list):
    """Sum all income amounts."""
    total = sum(item.get("amount", 0) for item in income_list)
    return total

def compute_total_expenses(expenses):
    """Compute total expenses for 'needs' and 'wants' separately and combined."""
    total_needs = sum(item.get("amount", 0) for item in expenses.get("needs", []))
    total_wants = sum(item.get("amount", 0) for item in expenses.get("wants", []))
    return total_needs, total_wants, total_needs + total_wants

def compute_debt_summary(debts):
    """Compute total debt and list of debts with computed monthly interest."""
    total_debt = 0
    detailed_debts = []
    for d in debts:
        amount = d.get("total_amount", 0)
        monthly_payment = d.get("monthly_payment", 0)
        apr = d.get("apr", 0)
        tenure = d.get("tenure", 0)  # in months
        monthly_interest_rate = apr / 12
        monthly_interest = amount * monthly_interest_rate
        total_debt += amount
        detailed_debts.append({
            "name": d.get("name", ""),
            "total_amount": amount,
            "monthly_payment": monthly_payment,
            "apr": apr,
            "tenure": tenure,
            "monthly_interest": monthly_interest
        })
    return total_debt, detailed_debts

##############################################
# Debt Repayment Simulations (Different Strategies)
##############################################

def simulate_debt_repayment_avalanche(debts, extra_payment):
    """
    Simulate the debt avalanche strategy (pay highest APR first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    # Sort debts by descending APR (highest interest first)
    debts = sorted(copy.deepcopy(debts), key=lambda d: d.get("apr", 0), reverse=True)
    total_interest = 0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        # Compute total minimum payments for this month
        total_min_payment = sum(min(d["monthly_payment"], d["total_amount"]) for d in debts if d["total_amount"] > 0)
        available_extra = extra_payment
        # First, pay minimum payments on all debts
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] += interest
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] -= pay
        # Then, allocate extra funds to the debt with the highest APR that still has balance.
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] -= payment
                available_extra -= payment
                if available_extra <= 0:
                    break
    return months, total_interest

def simulate_debt_repayment_snowball(debts, extra_payment):
    """
    Simulate the debt snowball strategy (pay smallest balance first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    debts = sorted(copy.deepcopy(debts), key=lambda d: d["total_amount"])
    total_interest = 0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        total_min_payment = sum(min(d["monthly_payment"], d["total_amount"]) for d in debts if d["total_amount"] > 0)
        available_extra = extra_payment
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] += interest
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] -= pay
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] -= payment
                available_extra -= payment
                if available_extra <= 0:
                    break
        debts = sorted(debts, key=lambda d: d["total_amount"])
    return months, total_interest

def simulate_debt_consolidation(debts, consolidation_rate, consolidation_term):
    """
    Calculate a consolidated monthly payment and simulate repayment.
    :param debts: List of debt dictionaries.
    :param consolidation_rate: New APR for consolidation.
    :param consolidation_term: Term in months.
    :return: Consolidated monthly payment, total interest paid over term.
    """
    total_balance = sum(d["total_amount"] for d in debts)
    monthly_rate = consolidation_rate / 12
    # Loan amortization formula
    payment = total_balance * (monthly_rate * (1 + monthly_rate) ** consolidation_term) / ((1 + monthly_rate) ** consolidation_term - 1)
    total_interest = 0
    remaining_balance = total_balance
    for _ in range(consolidation_term):
        interest = remaining_balance * monthly_rate
        total_interest += interest
        remaining_balance += interest - payment
        if remaining_balance < 0:
            remaining_balance = 0
            break
    return payment, total_interest

########################################
# Savings Projection (Compound Growth)
########################################

def project_savings_growth(initial_savings, monthly_contribution, annual_rate, years):
    """
    Calculate future value of savings with monthly contributions and compound interest.
    """
    months = years * 12
    monthly_rate = annual_rate / 12
    future_value = initial_savings * ((1 + monthly_rate) ** months)
    # Sum contributions with compound interest
    for m in range(1, months + 1):
        future_value += monthly_contribution * ((1 + monthly_rate) ** (months - m))
    return future_value

#######################################
# Main Financial Module Functionality #
#######################################

def compute_financial_analysis(user_data):
    """
    Compute a comprehensive set of financial metrics and simulation results.
    
    Expected user_data format:
    {
      "income": [{"title": "Job", "amount": 5000}, {"title": "Side Hustle", "amount": 1000}],
      "debt": [
          {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
          {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
      ],
      "expenses": {
          "needs": [{"title": "Rent", "amount": 1500}, {"title": "Utilities", "amount": 300}],
          "wants": [{"title": "Dining Out", "amount": 200}, {"title": "Entertainment", "amount": 150}]
      },
      "savings": 2000
    }
    """
    # Compute totals for income and expenses
    total_income = compute_total_income(user_data.get("income", []))
    total_needs, total_wants, total_expenses = compute_total_expenses(user_data.get("expenses", {}))
    
    # Compute debt summary
    total_debt, detailed_debts = compute_debt_summary(user_data.get("debt", []))
    
    # Compute net cash flow (income minus expenses and minimum debt payments)
    # For simplicity, sum the minimum payments from the debt list
    total_min_debt_payments = sum(d.get("monthly_payment", 0) for d in user_data.get("debt", []))
    net_cash_flow = total_income - (total_expenses + total_min_debt_payments)
    
    # Assume a recommended savings rate (if net cash flow is positive, allocate a fixed percentage)
    recommended_savings_rate = 0.2  # 20%
    recommended_monthly_savings = max(net_cash_flow * recommended_savings_rate, 0)
    
    # Debt repayment simulations:
    # Extra funds available for debt repayment: any surplus beyond expenses and savings allocation.
    extra_funds = max(net_cash_flow - recommended_monthly_savings, 0)
    
    avalanche_months, avalanche_interest = simulate_debt_repayment_avalanche(user_data.get("debt", []), extra_funds)
    snowball_months, snowball_interest = simulate_debt_repayment_snowball(user_data.get("debt", []), extra_funds)
    
    # For consolidation, assume a consolidation rate (e.g., 8%) and term (e.g., 60 months)
    consolidation_rate = 0.08
    consolidation_term = 60
    consolidation_payment, consolidation_interest = simulate_debt_consolidation(user_data.get("debt", []), consolidation_rate, consolidation_term)
    
    # Savings projection (assume an annual return rate, e.g., 4% for a savings account)
    annual_savings_rate = 0.04
    projected_savings_5_years = project_savings_growth(user_data.get("savings", 0) + recommended_monthly_savings, recommended_monthly_savings, annual_savings_rate, 5)
    
    analysis = {
        "Financial Summary": {
            "Total Income": total_income,
            "Total Expenses": total_expenses,
            "  Needs": total_needs,
            "  Wants": total_wants,
            "Total Debt": total_debt,
            "Current Savings": user_data.get("savings", 0),
            "Net Cash Flow": net_cash_flow,
            "Recommended Monthly Savings": recommended_monthly_savings
        },
        "Debt Details": detailed_debts,
        "Debt Repayment Simulations": {
            "Avalanche Strategy": {
                "Estimated Months to Debt-Free": avalanche_months,
                "Total Interest Paid": avalanche_interest,
                "Extra Funds Used Monthly": extra_funds
            },
            "Snowball Strategy": {
                "Estimated Months to Debt-Free": snowball_months,
                "Total Interest Paid": snowball_interest,
                "Extra Funds Used Monthly": extra_funds
            },
            "Consolidation Strategy": {
                "Monthly Consolidated Payment": consolidation_payment,
                "Total Interest Over Term": consolidation_interest,
                "Term (months)": consolidation_term,
                "Assumed Consolidation APR": consolidation_rate
            }
        },
        "Savings Projection": {
            "Projected Savings in 5 Years": projected_savings_5_years,
            "Assumed Annual Savings Growth Rate": annual_savings_rate
        }
    }
    
    return analysis

##########################
# Example Usage of Module #
##########################

def round_floats(obj):
    """
    Recursively round all float values in a nested structure (dict or list) to 2 decimal places.
    """
    if isinstance(obj, dict):
        return {k: round_floats(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [round_floats(elem) for elem in obj]
    elif isinstance(obj, float):
        return round(obj, 2)
    else:
        return obj

if __name__ == "__main__":
    # Sample user data
    user_data = {
        "income": [
            {"title": "Job", "amount": 5000},
            {"title": "Side Hustle", "amount": 1000},
            {"title": "Investment", "amount": 500}
        ],
        "debt": [
            {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
            {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
        ],
        "expenses": {
            "needs": [
                {"title": "Rent", "amount": 1500},
                {"title": "Utilities", "amount": 300}
            ],
            "wants": [
                {"title": "Dining Out", "amount": 200},
                {"title": "Entertainment", "amount": 150}
            ]
        },
        "savings": 2000
    }
    
    analysis = compute_financial_analysis(user_data)
    
    # Round all float values to 2 decimal places.
    analysis = round_floats(analysis)
    
    # Convert analysis to a formatted JSON string and save in a variable.
    json_output = json.dumps(analysis, indent=2)
    
    # Optionally print the JSON output.
    print(json_output)

{
  "Financial Summary": {
    "Total Income": 6500,
    "Total Expenses": 2150,
    "  Needs": 1800,
    "  Wants": 350,
    "Total Debt": 15000,
    "Current Savings": 2000,
    "Net Cash Flow": 4050,
    "Recommended Monthly Savings": 810.0
  },
  "Debt Details": [
    {
      "name": "Credit Card",
      "total_amount": 5000,
      "monthly_payment": 100,
      "apr": 0.18,
      "tenure": 60,
      "monthly_interest": 75.0
    },
    {
      "name": "Car Loan",
      "total_amount": 10000,
      "monthly_payment": 200,
      "apr": 0.05,
      "tenure": 60,
      "monthly_interest": 41.666666666666664
    }
  ],
  "Debt Repayment Simulations": {
    "Avalanche Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.35777536882316,
      "Extra Funds Used Monthly": 3240.0
    },
    "Snowball Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.35777536882316,
      "Extra Funds Used Monthly": 3240.0
    },
    "Con

In [10]:
import copy
import math
import random
import json

#####################################
# Data Structures and Input Parsing #
#####################################

# Expected input format (example):
# {
#   "income": [{"title": "Job", "amount": 5000}, {"title": "Side Hustle", "amount": 1000}],
#   "debt": [
#       {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
#       {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
#   ],
#   "expenses": {
#       "needs": [{"title": "Rent", "amount": 1500}, {"title": "Utilities", "amount": 300}],
#       "wants": [{"title": "Dining Out", "amount": 200}, {"title": "Entertainment", "amount": 150}]
#   },
#   "savings": 2000
# }

###############################
# Basic Financial Calculations #
###############################

def compute_total_income(income_list):
    """Sum all income amounts."""
    total = sum(item.get("amount", 0) for item in income_list)
    return round(total, 2)

def compute_total_expenses(expenses):
    """Compute total expenses for 'needs' and 'wants' separately and combined."""
    total_needs = sum(item.get("amount", 0) for item in expenses.get("needs", []))
    total_wants = sum(item.get("amount", 0) for item in expenses.get("wants", []))
    return round(total_needs, 2), round(total_wants, 2), round(total_needs + total_wants, 2)

def compute_debt_summary(debts):
    """Compute total debt and list of debts with computed monthly interest."""
    total_debt = 0
    detailed_debts = []
    for d in debts:
        amount = d.get("total_amount", 0)
        monthly_payment = d.get("monthly_payment", 0)
        apr = d.get("apr", 0)
        tenure = d.get("tenure", 0)  # in months
        monthly_interest_rate = apr / 12
        monthly_interest = amount * monthly_interest_rate
        total_debt += amount
        detailed_debts.append({
            "name": d.get("name", ""),
            "total_amount": round(amount, 2),
            "monthly_payment": round(monthly_payment, 2),
            # Convert APR to percentage string
            "apr": f"{apr*100:.2f}%",
            "tenure": tenure,
            "monthly_interest": round(monthly_interest, 2)
        })
    return round(total_debt, 2), detailed_debts

##############################################
# Debt Repayment Simulations (Different Strategies)
##############################################

def simulate_debt_repayment_avalanche(debts, extra_payment):
    """
    Simulate the debt avalanche strategy (pay highest APR first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    debts = sorted(copy.deepcopy(debts), key=lambda d: d.get("apr", 0), reverse=True)
    total_interest = 0.0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        available_extra = extra_payment
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] = round(d["total_amount"] + interest, 2)
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] = round(d["total_amount"] - pay, 2)
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] = round(d["total_amount"] - payment, 2)
                available_extra -= payment
                if available_extra <= 0:
                    break
    return months, round(total_interest, 2)

def simulate_debt_repayment_snowball(debts, extra_payment):
    """
    Simulate the debt snowball strategy (pay smallest balance first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    debts = sorted(copy.deepcopy(debts), key=lambda d: d["total_amount"])
    total_interest = 0.0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        available_extra = extra_payment
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] = round(d["total_amount"] + interest, 2)
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] = round(d["total_amount"] - pay, 2)
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] = round(d["total_amount"] - payment, 2)
                available_extra -= payment
                if available_extra <= 0:
                    break
        debts = sorted(debts, key=lambda d: d["total_amount"])
    return months, round(total_interest, 2)

def simulate_debt_consolidation(debts, consolidation_rate, consolidation_term):
    """
    Calculate a consolidated monthly payment and simulate repayment.
    :param debts: List of debt dictionaries.
    :param consolidation_rate: New APR for consolidation.
    :param consolidation_term: Term in months.
    :return: Consolidated monthly payment, total interest paid over term.
    """
    total_balance = sum(d["total_amount"] for d in debts)
    monthly_rate = consolidation_rate / 12
    payment = total_balance * (monthly_rate * (1 + monthly_rate) ** consolidation_term) / ((1 + monthly_rate) ** consolidation_term - 1)
    total_interest = 0.0
    remaining_balance = total_balance
    for _ in range(consolidation_term):
        interest = remaining_balance * monthly_rate
        total_interest += interest
        remaining_balance += interest - payment
        if remaining_balance < 0:
            remaining_balance = 0
            break
    return round(payment, 2), round(total_interest, 2)

########################################
# Savings Projection (Compound Growth)
########################################

def project_savings_growth(initial_savings, monthly_contribution, annual_rate, years):
    """
    Calculate future value of savings with monthly contributions and compound interest.
    """
    months = years * 12
    monthly_rate = annual_rate / 12
    future_value = initial_savings * ((1 + monthly_rate) ** months)
    for m in range(1, months + 1):
        future_value += monthly_contribution * ((1 + monthly_rate) ** (months - m))
    return round(future_value, 2)

#######################################
# Main Financial Module Functionality #
#######################################

def compute_financial_analysis(user_data):
    """
    Compute a comprehensive set of financial metrics and simulation results.
    
    Expected user_data format:
    {
      "income": [{"title": "Job", "amount": 5000}, {"title": "Side Hustle", "amount": 1000}],
      "debt": [
          {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
          {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
      ],
      "expenses": {
          "needs": [{"title": "Rent", "amount": 1500}, {"title": "Utilities", "amount": 300}],
          "wants": [{"title": "Dining Out", "amount": 200}, {"title": "Entertainment", "amount": 150}]
      },
      "savings": 2000
    }
    """
    total_income = compute_total_income(user_data.get("income", []))
    total_needs, total_wants, total_expenses = compute_total_expenses(user_data.get("expenses", {}))
    total_debt, detailed_debts = compute_debt_summary(user_data.get("debt", []))
    total_min_debt_payments = sum(d.get("monthly_payment", 0) for d in user_data.get("debt", []))
    net_cash_flow = total_income - (total_expenses + total_min_debt_payments)
    recommended_savings_rate = 0.2  # 20%
    recommended_monthly_savings = max(net_cash_flow * recommended_savings_rate, 0)
    extra_funds = max(net_cash_flow - recommended_monthly_savings, 0)
    
    avalanche_months, avalanche_interest = simulate_debt_repayment_avalanche(user_data.get("debt", []), extra_funds)
    snowball_months, snowball_interest = simulate_debt_repayment_snowball(user_data.get("debt", []), extra_funds)
    
    consolidation_rate = 0.08  # 8%
    consolidation_term = 60
    consolidation_payment, consolidation_interest = simulate_debt_consolidation(user_data.get("debt", []), consolidation_rate, consolidation_term)
    
    annual_savings_rate = 0.04  # 4%
    projected_savings_5_years = project_savings_growth(user_data.get("savings", 0) + recommended_monthly_savings, recommended_monthly_savings, annual_savings_rate, 5)
    
    analysis = {
        "Financial Summary": {
            "Total Income": total_income,
            "Total Expenses": total_expenses,
            "Needs": total_needs,
            "Wants": total_wants,
            "Total Debt": total_debt,
            "Current Savings": round(user_data.get("savings", 0), 2),
            "Net Cash Flow": net_cash_flow,
            "Recommended Monthly Savings": recommended_monthly_savings
        },
        "Debt Details": detailed_debts,
        "Debt Repayment Simulations": {
            "Avalanche Strategy": {
                "Estimated Months to Debt-Free": avalanche_months,
                "Total Interest Paid": avalanche_interest,
                "Extra Funds Used Monthly": round(extra_funds, 2)
            },
            "Snowball Strategy": {
                "Estimated Months to Debt-Free": snowball_months,
                "Total Interest Paid": snowball_interest,
                "Extra Funds Used Monthly": round(extra_funds, 2)
            },
            "Consolidation Strategy": {
                "Monthly Consolidated Payment": consolidation_payment,
                "Total Interest Over Term": consolidation_interest,
                "Term (months)": consolidation_term,
                "Assumed Consolidation APR": f"{consolidation_rate*100:.2f}%"
            }
        },
        "Savings Projection": {
            "Projected Savings in 5 Years": projected_savings_5_years,
            "Assumed Annual Savings Growth Rate": f"{annual_savings_rate*100:.2f}%"
        }
    }
    
    return analysis

#######################################
# Example Usage of Module and JSON Save #
#######################################

if __name__ == "__main__":
    user_data = {
        "income": [
            {"title": "Job", "amount": 5000},
            {"title": "Side Hustle", "amount": 1000},
            {"title": "Investment", "amount": 500}
        ],
        "debt": [
            {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
            {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
        ],
        "expenses": {
            "needs": [
                {"title": "Rent", "amount": 1500},
                {"title": "Utilities", "amount": 300}
            ],
            "wants": [
                {"title": "Dining Out", "amount": 200},
                {"title": "Entertainment", "amount": 150}
            ]
        },
        "savings": 2000
    }
    
    analysis = compute_financial_analysis(user_data)
    
    # Convert analysis to a formatted JSON string and save in a variable.
    json_output = json.dumps(analysis, indent=2)
    
    print(json_output)


{
  "Financial Summary": {
    "Total Income": 6500,
    "Total Expenses": 2150,
    "Needs": 1800,
    "Wants": 350,
    "Total Debt": 15000,
    "Current Savings": 2000,
    "Net Cash Flow": 4050,
    "Recommended Monthly Savings": 810.0
  },
  "Debt Details": [
    {
      "name": "Credit Card",
      "total_amount": 5000,
      "monthly_payment": 100,
      "apr": "18.00%",
      "tenure": 60,
      "monthly_interest": 75.0
    },
    {
      "name": "Car Loan",
      "total_amount": 10000,
      "monthly_payment": 200,
      "apr": "5.00%",
      "tenure": 60,
      "monthly_interest": 41.67
    }
  ],
  "Debt Repayment Simulations": {
    "Avalanche Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.36,
      "Extra Funds Used Monthly": 3240.0
    },
    "Snowball Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.36,
      "Extra Funds Used Monthly": 3240.0
    },
    "Consolidation Strategy": {
      "Mon

## Generate Financial Report

In [3]:
import copy
import math
import random
import json

###############################
# Basic Financial Calculations #
###############################

def compute_total_income(income_list):
    """Sum all income amounts."""
    total = sum(item.get("amount", 0) for item in income_list)
    return total

def compute_total_expenses(expenses):
    """Compute total expenses for 'needs' and 'wants' separately and combined."""
    total_needs = sum(item.get("amount", 0) for item in expenses.get("needs", []))
    total_wants = sum(item.get("amount", 0) for item in expenses.get("wants", []))
    return total_needs, total_wants, total_needs + total_wants

def compute_debt_summary(debts):
    """Compute total debt and list of debts with computed monthly interest."""
    total_debt = 0
    detailed_debts = []
    for d in debts:
        amount = d.get("total_amount", 0)
        monthly_payment = d.get("monthly_payment", 0)
        apr = d.get("apr", 0)
        tenure = d.get("tenure", 0)  # in months
        monthly_interest_rate = apr / 12
        monthly_interest = amount * monthly_interest_rate
        total_debt += amount
        detailed_debts.append({
            "name": d.get("name", ""),
            "total_amount": amount,
            "monthly_payment": monthly_payment,
            "apr": apr,
            "tenure": tenure,
            "monthly_interest": monthly_interest
        })
    return total_debt, detailed_debts

##############################################
# Debt Repayment Simulations (Different Strategies)
##############################################

def simulate_debt_repayment_avalanche(debts, extra_payment):
    """
    Simulate the debt avalanche strategy (pay highest APR first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    # Sort debts by descending APR (highest interest first)
    debts = sorted(copy.deepcopy(debts), key=lambda d: d.get("apr", 0), reverse=True)
    total_interest = 0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        total_min_payment = sum(min(d["monthly_payment"], d["total_amount"]) for d in debts if d["total_amount"] > 0)
        available_extra = extra_payment
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] += interest
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] -= pay
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] -= payment
                available_extra -= payment
                if available_extra <= 0:
                    break
    return months, total_interest

def simulate_debt_repayment_snowball(debts, extra_payment):
    """
    Simulate the debt snowball strategy (pay smallest balance first).
    :param debts: List of debt dictionaries (copy them before simulation).
    :param extra_payment: Extra funds available each month to pay towards debt.
    :return: Total months required and total interest paid.
    """
    debts = sorted(copy.deepcopy(debts), key=lambda d: d["total_amount"])
    total_interest = 0
    months = 0

    while any(d["total_amount"] > 0 for d in debts):
        months += 1
        total_min_payment = sum(min(d["monthly_payment"], d["total_amount"]) for d in debts if d["total_amount"] > 0)
        available_extra = extra_payment
        for d in debts:
            if d["total_amount"] <= 0:
                continue
            monthly_rate = d["apr"] / 12
            interest = d["total_amount"] * monthly_rate
            total_interest += interest
            d["total_amount"] += interest
            pay = min(d["monthly_payment"], d["total_amount"])
            d["total_amount"] -= pay
        for d in debts:
            if d["total_amount"] > 0 and available_extra > 0:
                payment = min(available_extra, d["total_amount"])
                d["total_amount"] -= payment
                available_extra -= payment
                if available_extra <= 0:
                    break
        debts = sorted(debts, key=lambda d: d["total_amount"])
    return months, total_interest

def simulate_debt_consolidation(debts, consolidation_rate, consolidation_term):
    """
    Calculate a consolidated monthly payment and simulate repayment.
    :param debts: List of debt dictionaries.
    :param consolidation_rate: New APR for consolidation.
    :param consolidation_term: Term in months.
    :return: Consolidated monthly payment, total interest paid over term.
    """
    total_balance = sum(d["total_amount"] for d in debts)
    monthly_rate = consolidation_rate / 12
    payment = total_balance * (monthly_rate * (1 + monthly_rate) ** consolidation_term) / ((1 + monthly_rate) ** consolidation_term - 1)
    total_interest = 0
    remaining_balance = total_balance
    for _ in range(consolidation_term):
        interest = remaining_balance * monthly_rate
        total_interest += interest
        remaining_balance += interest - payment
        if remaining_balance < 0:
            remaining_balance = 0
            break
    return payment, total_interest

########################################
# Savings Projection (Compound Growth)
########################################

def project_savings_growth(initial_savings, monthly_contribution, annual_rate, years):
    """
    Calculate future value of savings with monthly contributions and compound interest.
    """
    months = years * 12
    monthly_rate = annual_rate / 12
    future_value = initial_savings * ((1 + monthly_rate) ** months)
    for m in range(1, months + 1):
        future_value += monthly_contribution * ((1 + monthly_rate) ** (months - m))
    return future_value

#######################################
# Main Financial Module Functionality #
#######################################

def compute_financial_analysis(user_data):
    """
    Compute a comprehensive set of financial metrics and simulation results.
    
    Expected user_data format:
    {
      "income": [{"title": "Job", "amount": 5000}, {"title": "Side Hustle", "amount": 1000}],
      "debt": [
          {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
          {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
      ],
      "expenses": {
          "needs": [{"title": "Rent", "amount": 1500}, {"title": "Utilities", "amount": 300}],
          "wants": [{"title": "Dining Out", "amount": 200}, {"title": "Entertainment", "amount": 150}]
      },
      "savings": 2000
    }
    """
    total_income = compute_total_income(user_data.get("income", []))
    total_needs, total_wants, total_expenses = compute_total_expenses(user_data.get("expenses", {}))
    
    total_debt, detailed_debts = compute_debt_summary(user_data.get("debt", []))
    
    total_min_debt_payments = sum(d.get("monthly_payment", 0) for d in user_data.get("debt", []))
    net_cash_flow = total_income - (total_expenses + total_min_debt_payments)
    
    recommended_savings_rate = 0.2  # 20%
    recommended_monthly_savings = max(net_cash_flow * recommended_savings_rate, 0)
    
    extra_funds = max(net_cash_flow - recommended_monthly_savings, 0)
    
    avalanche_months, avalanche_interest = simulate_debt_repayment_avalanche(user_data.get("debt", []), extra_funds)
    snowball_months, snowball_interest = simulate_debt_repayment_snowball(user_data.get("debt", []), extra_funds)
    
    consolidation_rate = 0.08
    consolidation_term = 60
    consolidation_payment, consolidation_interest = simulate_debt_consolidation(user_data.get("debt", []), consolidation_rate, consolidation_term)
    
    annual_savings_rate = 0.04
    projected_savings_5_years = project_savings_growth(user_data.get("savings", 0) + recommended_monthly_savings, recommended_monthly_savings, annual_savings_rate, 5)
    
    analysis = {
        "Financial Summary": {
            "Total Income": total_income,
            "Total Expenses": total_expenses,
            "  Needs": total_needs,
            "  Wants": total_wants,
            "Total Debt": total_debt,
            "Current Savings": user_data.get("savings", 0),
            "Net Cash Flow": net_cash_flow,
            "Recommended Monthly Savings": recommended_monthly_savings
        },
        "Debt Details": detailed_debts,
        "Debt Repayment Simulations": {
            "Avalanche Strategy": {
                "Estimated Months to Debt-Free": avalanche_months,
                "Total Interest Paid": avalanche_interest,
                "Extra Funds Used Monthly": extra_funds
            },
            "Snowball Strategy": {
                "Estimated Months to Debt-Free": snowball_months,
                "Total Interest Paid": snowball_interest,
                "Extra Funds Used Monthly": extra_funds
            },
            "Consolidation Strategy": {
                "Monthly Consolidated Payment": consolidation_payment,
                "Total Interest Over Term": consolidation_interest,
                "Term (months)": consolidation_term,
                "Assumed Consolidation APR": consolidation_rate
            }
        },
        "Savings Projection": {
            "Projected Savings in 5 Years": projected_savings_5_years,
            "Assumed Annual Savings Growth Rate": annual_savings_rate
        }
    }
    
    return analysis

##########################
# Example Usage of Module #
##########################

def round_floats(obj):
    """
    Recursively round all float values in a nested structure (dict or list) to 2 decimal places.
    """
    if isinstance(obj, dict):
        return {k: round_floats(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [round_floats(elem) for elem in obj]
    elif isinstance(obj, float):
        return round(obj, 2)
    else:
        return obj
    
    
from langchain_ollama import OllamaLLM

def generate_financial_report_llama(financial_data):
    """
    Generate a detailed financial report using a local LLaMA model via LangChain's LlamaCpp.
    The prompt includes the full financial analysis data to allow the model to generate an in-depth report.
    """
    # Expand the prompt as before; you can adjust the structure as needed.
    prompt = f"""
You are a seasoned financial advisor with expertise in personal finance and wealth management.
Generate a detailed, data-driven financial report using the following computed financial analysis.
Use the exact numbers and computed values to provide specific, actionable recommendations.
Ensure that your report includes:
1. An Introduction summarizing the user's financial situation (total income, expenses, debt, current savings).
2. A Debt Repayment Plan detailing each debt, extra payments, estimated time to become debt-free, and total interest saved.
3. A Current Savings Status section that explains current savings and recommended monthly savings.
4. Future Savings Projections and Investment Recommendations with precise 5-year growth projections.
5. A Conclusion that recaps the key recommendations and provides clear next steps.

Financial Analysis Data:
{json.dumps(financial_data, indent=2)}

Please produce a comprehensive report of about 500 words that includes exact values, percentage splits, and clear, step-by-step recommendations.
"""
    # Instantiate the LlamaCpp model. Adjust the model_path to your local LLaMA model file.
    llm = OllamaLLM(model="llama3.2", prompt=prompt)
    report = llm(prompt)
    return report


In [4]:
if __name__ == "__main__":
    user_data = {
        "income": [
            {"title": "Job", "amount": 5000},
            {"title": "Side Hustle", "amount": 1000},
            {"title": "Investment", "amount": 500}
        ],
        "debt": [
            {"name": "Credit Card", "total_amount": 5000, "monthly_payment": 100, "apr": 0.18, "tenure": 60},
            {"name": "Car Loan", "total_amount": 10000, "monthly_payment": 200, "apr": 0.05, "tenure": 60}
        ],
        "expenses": {
            "needs": [
                {"title": "Rent", "amount": 1500},
                {"title": "Utilities", "amount": 300}
            ],
            "wants": [
                {"title": "Dining Out", "amount": 200},
                {"title": "Entertainment", "amount": 150}
            ]
        },
        "savings": 2000
    }
    
    analysis = compute_financial_analysis(user_data)
    # Optionally, you can round all values here if not already done.
    json_output = json.dumps(analysis, indent=2)
    print("Computed Financial Analysis:")
    print(json_output)
    
    try:
        # Use the LLaMA model via LangChain to generate the report.
        report = generate_financial_report_llama(analysis)
        print("\nGenerated Financial Report:")
        print(report)
    except Exception as e:
        print(f"Error generating report: {e}")


Computed Financial Analysis:
{
  "Financial Summary": {
    "Total Income": 6500,
    "Total Expenses": 2150,
    "  Needs": 1800,
    "  Wants": 350,
    "Total Debt": 15000,
    "Current Savings": 2000,
    "Net Cash Flow": 4050,
    "Recommended Monthly Savings": 810.0
  },
  "Debt Details": [
    {
      "name": "Credit Card",
      "total_amount": 5000,
      "monthly_payment": 100,
      "apr": 0.18,
      "tenure": 60,
      "monthly_interest": 75.0
    },
    {
      "name": "Car Loan",
      "total_amount": 10000,
      "monthly_payment": 200,
      "apr": 0.05,
      "tenure": 60,
      "monthly_interest": 41.666666666666664
    }
  ],
  "Debt Repayment Simulations": {
    "Avalanche Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.35777536882316,
      "Extra Funds Used Monthly": 3240.0
    },
    "Snowball Strategy": {
      "Estimated Months to Debt-Free": 5,
      "Total Interest Paid": 242.35777536882316,
      "Extra Funds Used Mon